In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import experiment_helpers as eh
import train

In [ ]:
meta = eh.Metadata()
meta._metadata

In [ ]:
# Eager execution makes the training and execution ~25x slower.
tf.compat.v1.disable_eager_execution()

In [ ]:
layers = [[18, 'sigmoid'],
          [9, 'linear']]

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(9,)))
for layer in layers:
    model.add(tf.keras.layers.Dense(layer[0], activation=layer[1]))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])
model.summary()

In [ ]:
num_games = 10000
start_time = time.time()
(outcomes, metadata) = train.q_learning(model, num_games)
end_time = time.time()
total_time = end_time - start_time
print("total time:", total_time)

metadata['training_time'] = total_time
metadata['layers'] = layers

In [ ]:
(invalid, loss, tie, win) = eh.calculate_training_sequence(outcomes)

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1)
x_length = len(invalid)
x_vals = np.array(range(x_length)) * 100 / x_length
ax.plot(x_vals, invalid, 'r')
ax.plot(x_vals, loss, 'k')
ax.plot(x_vals, tie, 'b')
ax.plot(x_vals, win, 'g')
ax.set_ylim(-0.01, 1.01)
ax.set_title('Game outcomes during training', fontsize=20)
ax.set_xlabel('Percent of training completed', fontsize=18)
ax.set_ylabel('Portion of each outcome', fontsize=18)
plt.show()

In [ ]:
# Calculate head-to-head stats vs a random player.
opponent = 'random'
stats = eh.head_to_head(model, opponent)
metadata['stats_{}'.format(opponent)] = stats
stats

In [ ]:
# Get the opening move that this model would make on a blank board.
position = eh.evaluate_opening_move(model)
metadata['opening_move'] = [int(position[0]), int(position[1])]  # Convert to support JSON serialization.
position

In [ ]:
metadata

In [ ]:
# Add the new metadata.
version = meta.add_experiment(metadata)
meta.exploded_copy()

In [ ]:
# Save the metadata
meta.save()

In [ ]:
# Save the model.
# model_directory = 'model_v{}'.format(version)
# model.save(model_directory)